In [1]:
import cv2
import numpy as np
import os
from PIL import Image
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from torchvision.utils import save_image
import time
from tqdm import tqdm
from models import *
from utils import *
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/u9564043/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_quality = [40]
num_imgs = 0
mode = 1
if mode == 0:
    quality_subs =   ['ISR', 'IQE','ISR + IQE', 'IQE + ISR', 'IQE + ISR + IQE', 'e2e']
    detect_subs = ['','_ISR','_IQE', '_ISR + IQE', '_IQE + ISR', '_IQE + ISR + IQE', '_e2e', 'full']
elif mode == 1:
    quality_subs = ['IQE','ISR + IQE', 'IQE + ISR', 'IQE + ISR + IQE', 'SRQE', 'e2e']
    detect_subs = ['_IQE', '_ISR + IQE', '_IQE + ISR', '_IQE + ISR + IQE', '_SRQE','_e2e']
elif mode == 2:
    quality_subs = ['IQE','ISR + IQE', 'IQE + ISR', 'IQE + ISR + IQE']
    detect_subs = ['_IQE', '_ISR + IQE', '_IQE + ISR', '_IQE + ISR + IQE']
elif mode == 3:
    quality_subs =  ['ISR', 'IQE','ISR + IQE', 'IQE + ISR', 'IQE + ISR + IQE']
    detect_subs = ['_ISR','_IQE', '_ISR + IQE', '_IQE + ISR', '_IQE + ISR + IQE']
elif mode == 4:
    quality_subs =  ['IQE']
    detect_subs = ['_IQE']

In [3]:
# img_dir = '/work/u9564043/Machine-Vision/dataset/PCB_dataset/test/images'
# psnr_dict = {
#             "mouse_bite" :[],
#             "spur_":[], 
#             "missing_hole":[],
#             "short":[],
#             "open_circuit":[],
#             "spurious_copper":[]
    
#     }

# hr_img_path = f'output/test_600/images'
# hr_label_path = f'output/test_600/labels'

# os.makedirs(hr_img_path, exist_ok = True)
# os.makedirs(hr_label_path, exist_ok = True)

# max_per_label = num_imgs
# label_counter = {k: 0 for k in psnr_dict.keys()}
# # os.makedirs(output, exist_ok = True)
# for path in tqdm(os.listdir(img_dir), unit = 'img'):
#     label = None
#     for key in psnr_dict.keys():
#         if key in path:  # Nếu tên file chứa tên nhãn
#             label = key
#             break
#     if label is None:
#         continue  # Bỏ qua nếu không xác định được nhãn

#     # --- Giới hạn tối đa 100 ảnh mỗi nhãn ---
#     if num_imgs != 0:
#         if label_counter[label] >= max_per_label:
#             continue
#         label_counter[label] += 1
    
#     source_file = os.path.join(img_dir, path)
#     file1 = os.path.join(hr_img_path, path)
#     shutil.copy(source_file, file1)

# source_dir = f'/work/u9564043/Machine-Vision/dataset/PCB_dataset/test/labels'
# # Tạo thư mục đích nếu chưa tồn tại

# # Sao chép các file từ thư mục nguồn sang thư mục đích
# for filename in tqdm(os.listdir(source_dir)):
#     source_file = os.path.join(source_dir, filename)
#     file2 = os.path.join(hr_label_path, filename)
#     shutil.copy(source_file, file2)
# print("Đã sao chép các file thành công.")

In [4]:
# img_dir = hr_img_path
# psnr_dict = {
#             "mouse_bite" :[],
#             "spur_":[], 
#             "missing_hole":[],
#             "short":[],
#             "open_circuit":[],
#             "spurious_copper":[]
    
#     }
# ssim_dict = {
#         "mouse_bite" :[],
#         "spur_":[], 
#         "missing_hole":[],
#         "short":[],
#         "open_circuit":[],
#         "spurious_copper":[]

#     }


# # for quality in [10, 20, 30, 40, 80]:
# for quality in test_quality:
#     types = f'test_{quality}'
#     # output = f'compress/quality_compress_{quality}'
#     output_lr_img = f'output/{types}_150/images'
#     output_hr_img = f'output/{types}_600/images'
#     output_lr_label = f'output/{types}_150/labels'
#     output_hr_label = f'output/{types}_600/labels'
    
#     os.makedirs(output_lr_img, exist_ok = True)
#     os.makedirs(output_hr_img, exist_ok = True)
#     os.makedirs(output_lr_label, exist_ok = True)
#     os.makedirs(output_hr_label, exist_ok = True)

#     max_per_label = 100
#     label_counter = {k: 0 for k in psnr_dict.keys()}
#     # os.makedirs(output, exist_ok = True)
#     for path in tqdm(os.listdir(img_dir), desc = f'quality {quality}', unit = 'img'):
        
#         img_path = os.path.join(img_dir, path)
#         # out_path = os.path.join(output, f"{os.path.splitext(path)[0]}.png")
#         out_lr_path = os.path.join(output_lr_img, f"{os.path.splitext(path)[0]}.png")
#         out_hr_path = os.path.join(output_hr_img, f"{os.path.splitext(path)[0]}.png")
#         img = Image.open(img_path)
#         img = img.resize((608, 608))
#         w, h = img.size
#         # print(h, w)
#         out = img.resize((w//4, h//4))
#         # out = img
#         if quality == 'origin':
#             out.save(out_lr_path, format="PNG", optimize=True)
            
#         else:
#             out.save(out_lr_path, format="JPEG", quality=quality, optimize=True)
#         img.save(out_hr_path, format='JPEG', quality=quality, optimize=True)
    
#     source_dir = hr_label_path
#     # Tạo thư mục đích nếu chưa tồn tại
    
#     # Sao chép các file từ thư mục nguồn sang thư mục đích
#     for filename in tqdm(os.listdir(source_dir)):
#         source_file = os.path.join(source_dir, filename)
#         file1 = os.path.join(output_lr_label, filename)
#         file2 = os.path.join(output_hr_label, filename)
#         shutil.copy(source_file, file1)
#         shutil.copy(source_file, file2)
#     print("Đã sao chép các file thành công.")

In [5]:
hr_img_path = f'output/test_600/images'
hr_label_path = f'output/test_600/labels'
iqe_types = 'small'
imgsz = 64
# iqe = IQE().to(device)
if iqe_types=='small':
    iqe = Enhancer().to(device)
    iqe_sr = Enhancer().to(device)
    iqe_e2e = Enhancer().to(device)
    iqe_srqe = Enhancer().to(device)
else:
    iqe = Enhancer(in_nc=3, out_nc=3,nf=64, level=2, num_blocks=[2, 4, 4]).to(device)
    iqe_sr = Enhancer(in_nc=3, out_nc=3,nf=64, level=2, num_blocks=[2, 4, 4]).to(device)
    iqe_e2e = Enhancer(in_nc=3, out_nc=3,nf=64, level=2, num_blocks=[2, 4, 4]).to(device)
    iqe_srqe = Enhancer(in_nc=3, out_nc=3,nf=64, level=2, num_blocks=[2, 4, 4]).to(device)

isr = ESR(scale_factor=4, use_canny=True).to(device)
# iqe = SwinIR(upscale=1, in_chans=3, img_size=126, window_size=7,img_range=255., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6], mlp_ratio=2, upsampler='', resi_connection='1conv').to(device)
# isr = Interpolate(scale_factor=4, mode='bicubic').to(device)

# iqe_sr = IQE().to(device)

isr_qe = ESR(scale_factor=4, use_canny=True).to(device)

isr.load_state_dict(torch.load(f'exp/weights/best_esr_40.pth', map_location=device)['model_state_dict'])
iqe.load_state_dict(torch.load(f'exp/bestweight/best_weight_enhance_{iqe_types}_lr_{imgsz}.pth', map_location=device)['model_state_dict'])

# iqe_sr.load_state_dict(torch.load(f'exp/bestweight/best_weight_enhance_{iqe_types}_isrout_{imgsz}.pth', map_location=device)['model_state_dict'])
# iqe_sr.load_state_dict(torch.load(f'exp/Enhancer_Small_e2e_64/best_iqe_weight.pth', map_location=device)['model_state_dict'])
iqe_sr.load_state_dict(torch.load(f'exp/Enhancer_Small_ISRout_64/best_weight.pth', map_location=device)['model_state_dict'])
iqe_srqe.load_state_dict(torch.load(f'exp/Enhancer_Small_SRQE_64/best_weight.pth', map_location=device)['model_state_dict'])
iqe_e2e.load_state_dict(torch.load(f'exp/bestweight/best_weight_enhance_small_e2e_fpnloss.pth', map_location=device)['model_state_dict'])

isr_qe.load_state_dict(torch.load(f'exp/weights/best_esr_40.pth', map_location=device)['model_state_dict'])
# iqe_e2e = iqe_sr


<ipython-input-5-86e2cbe7a636>:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  isr.load_state_dict(torch.load(f'exp/weights/best_esr_40.pth', map_location=device)['model_s

<All keys matched successfully>

In [6]:
from torchmetrics import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure

def calculate_metrics(img1, img2, max_pixel_value=1.0):
    psnr = PeakSignalNoiseRatio(data_range=1.0).to(device)
    ssim = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)
    psnr_value = psnr(img1, img2)
    ssim_value = ssim(img1, img2)
    return psnr_value, ssim_value



In [7]:
# # for (sub, model) in zip(['LR', 'Bicubic','HQSR_C', 'HQSR_S', 'EDSR', 'SRResNet', 'VDSR', 'SRCNN', 'HR'], [lr, bicubic, esr_canny, esr_sobel,edsr,  srresnet, vdsr, srcnn, hr]):
# # for (sub, model) in zip([ 'ISR', 'IQE', 'ISR + IQE', 'IQE + ISR', 'IQE + ISR + IQE'], [, esr_sobel]):
# # for quality in [10, 20, 40, 80]:
# # for quality in [10, 20, 40, 30]:
# for quality in test_quality:
#     # for sub in ['ISR', 'IQE','ISR + IQE', 'IQE + ISR', 'IQE + ISR + IQE']:
#     # for sub in ['ISR', 'IQE','IQE + ISR', 'ISR + IQE']:
#     # for sub in ['ISR + IQE' ]:
#     for sub in ['SRQE']:
#     # iqe_sr = iqe
#     # isr_qe = isr
#     # for sub in quality_subs:
#         for part in ['train', 'test']:
#             print(f"Running {sub} on quality {quality}...")
#             hr_image_dir = f'dataset/40_e2e/{part}/LQ'
#             lr_image_dir = f'dataset/40_e2e/{part}/LQ'
#             # lr_image_dir = hr_image_dir
#             output_image_dir = f'dataset/40_e2e/{part}/SRQE'
#             os.makedirs(output_image_dir, exist_ok = True)
#             # Duyệt qua các ảnh trong thư mục
#             lr_image_files = os.listdir(lr_image_dir)
#             hr_image_files = os.listdir(hr_image_dir)
#             lr_image_files.sort()
#             hr_image_files.sort()
        
#             psnr_dict = {
#                 "mouse_bite" :[],
#                 "spur_":[], 
#                 "missing_hole":[],
#                 "short":[],
#                 "open_circuit":[],
#                 "spurious_copper":[]
        
#             }
#             ssim_dict = {
#                 "mouse_bite" :[],
#                 "spur_":[], 
#                 "missing_hole":[],
#                 "short":[],
#                 "open_circuit":[],
#                 "spurious_copper":[]
        
#             }
            
#             start = time.time()
#             transform = transforms.ToTensor()
#             print(lr_image_dir)
#             with torch.no_grad():
#                 for lr_image_file, hr_image_file in tqdm(zip(lr_image_files, hr_image_files), unit = 'img'):
#                     # Đường dẫn đến ảnh
                    
#                     lr_image_path = os.path.join(lr_image_dir, lr_image_file)
#                     hr_image_path = os.path.join(hr_image_dir, hr_image_file)
#                     output_image_path = os.path.join(output_image_dir, hr_image_file)
        
#                     # Tải và chuyển đổi ảnh
#                     lr_image = Image.open(lr_image_path).convert('RGB')
#                     hr_image = Image.open(hr_image_path).convert('RGB')
#                     hr_image = hr_image.resize((608, 608))
#                     lr_image = lr_image.resize((152, 152))
#                     if sub == 'IQE':
#                         hr_image = hr_image.resize((152, 152))
#                     # lr_image = Image.open(lr_image_path).convert('RGB')
        
#                     lr_image = transform(lr_image).unsqueeze(0).to(device)  # Thêm batch dimension và chuyển sang GPU
#                     hr_image = transform(hr_image).unsqueeze(0).to(device)  # Thêm batch dimension và chuyển sang GPU
#                     # if model == bicubic:
#                     # Dự đoán
#                     if sub == 'ISR':
#                         output = isr(lr_image)
#                     elif sub == 'IQE':
#                         output = iqe(lr_image)
#                     elif sub == 'ISR + IQE':
#                         output = iqe_sr(isr(lr_image))
#                     elif sub == 'IQE + ISR':
#                         output = isr(iqe(lr_image))
#                     elif sub == 'IQE + ISR + IQE':
#                         output = iqe_sr(isr_qe(iqe(lr_image)))
#                     elif sub == 'e2e':
#                         output = iqe_e2e(isr(lr_image))
#                     elif sub == 'SRQE':
#                         output = iqe_srqe(isr(lr_image))
#                     psnr,ssim = calculate_metrics(output, hr_image)
#                     for key in psnr_dict.keys():
#                         if key in lr_image_path:
#                             psnr_dict[key].append(psnr)
#                             ssim_dict[key].append(ssim)
#                             break

#                     # Chuyển đổi tensor đầu ra thành ảnh và lưu
#                     output_image = output.squeeze(0).to(device)  # Loại bỏ batch dimension và chuyển tensor sang CPU
#                     output_image = transforms.ToPILImage()(output_image)  # Chuyển tensor thành ảnh PIL
#                     output_image.save(output_image_path, )  # Lưu ảnh
#             avg_psnr = [0, 0, 0, 0, 0, 0]
#             avg_ssim = [0, 0, 0, 0, 0, 0]


#             # Tính toán PSNR trung bình
#             avg_psnr[0] = sum(psnr_dict['mouse_bite'])/len(psnr_dict['mouse_bite']) #mousebite_psnr
#             avg_psnr[1] = sum(psnr_dict['spur_'])/len(psnr_dict['spur_']) #spur_psnr
#             avg_psnr[2] = sum(psnr_dict['missing_hole'])/len(psnr_dict['missing_hole']) #missinghole_psnr 
#             avg_psnr[3] = sum(psnr_dict['short'])/len(psnr_dict['short']) #short_psnr
#             avg_psnr[4] = sum(psnr_dict['open_circuit'])/len(psnr_dict['open_circuit']) #opencircuit_psnr
#             avg_psnr[5]= sum(psnr_dict['spurious_copper'])/len(psnr_dict['spurious_copper']) #spuriouscopper_psnr 
#             average_psnr = sum(avg_psnr)/len(avg_psnr)

#             avg_ssim[0] = sum(ssim_dict['mouse_bite'])/len(ssim_dict['mouse_bite']) #mousebite_ssim
#             avg_ssim[1] = sum(ssim_dict['spur_'])/len(ssim_dict['spur_']) #spur_ssim
#             avg_ssim[2] = sum(ssim_dict['missing_hole'])/len(ssim_dict['missing_hole']) #missinghole_ssim 
#             avg_ssim[3] = sum(ssim_dict['short'])/len(ssim_dict['short']) #short_ssim
#             avg_ssim[4] = sum(ssim_dict['open_circuit'])/len(ssim_dict['open_circuit']) #opencircuit_ssim
#             avg_ssim[5]= sum(ssim_dict['spurious_copper'])/len(ssim_dict['spurious_copper']) #spuriouscopper_ssim  
#             average_ssim = sum(avg_ssim)/len(avg_ssim)
#             end = time.time()

#             with open('runs/enhance_results.txt', 'a') as f:
#                 f.write(output_image_dir.split('/')[1] + '\n')
#                 f.write(f'missinghole_psnr: {avg_psnr[2]:.2f}' + '\n')
#                 f.write(f'mousebite_psnr: {avg_psnr[0]:.2f}' + '\n')
#                 f.write(f'opencircuit_psnr: {avg_psnr[4]:.2f}' + '\n')
#                 f.write(f'short_psnr: {avg_psnr[3]:.2f}' + '\n')
#                 f.write(f'spur_psnr: {avg_psnr[1]:.2f}' + '\n')
#                 f.write(f'spuriouscopper_psnr: {avg_psnr[5]:.2f}' + '\n')
#                 f.write(f'average_psnr: {average_psnr:.2f}' + '\n')
#                 f.write(f'time process: {end - start:.2f}' + '\n')

#                 f.write(f'missinghole_psnr: {avg_ssim[2]:.4f}' + '\n')
#                 f.write(f'mousebite_psnr: {avg_ssim[0]:.4f}' + '\n')
#                 f.write(f'opencircuit_psnr: {avg_ssim[4]:.4f}' + '\n')
#                 f.write(f'short_psnr: {avg_ssim[3]:.4f}' + '\n')
#                 f.write(f'spur_psnr: {avg_ssim[1]:.4f}' + '\n')
#                 f.write(f'spuriouscopper_psnr: {avg_ssim[5]:.4f}' + '\n')
#                 f.write(f'average_psnr: {average_ssim:.4f}' + '\n')
#                 f.write(f'time process: {end - start:.4f}' + '\n')
#                 f.write('\n')
#                 f.flush()
#             source_dir = hr_label_path
        
            

In [8]:
# for (sub, model) in zip(['LR', 'Bicubic','HQSR_C', 'HQSR_S', 'EDSR', 'SRResNet', 'VDSR', 'SRCNN', 'HR'], [lr, bicubic, esr_canny, esr_sobel,edsr,  srresnet, vdsr, srcnn, hr]):
# for (sub, model) in zip([ 'ISR', 'IQE', 'ISR + IQE', 'IQE + ISR', 'IQE + ISR + IQE'], [, esr_sobel]):
# for quality in [10, 20, 40, 80]:
# for quality in [10, 20, 40, 30]:
for quality in test_quality:
    # for sub in ['ISR', 'IQE','ISR + IQE', 'IQE + ISR', 'IQE + ISR + IQE']:
    # for sub in ['ISR', 'IQE','IQE + ISR', 'ISR + IQE']:
    # for sub in ['ISR + IQE' ]:
    for sub in ['e2e']:
    # iqe_sr = iqe
    # isr_qe = isr
    # for sub in quality_subs:
        print(f"Running {sub} on quality {quality}...")
        hr_image_dir = hr_img_path
        lr_image_dir = f'output/test_{quality}_150/images'
        # lr_image_dir = hr_image_dir
        output_image_dir = f'output/enhance_{iqe_types}{imgsz}/test_{quality}_150_{sub}/images'
        os.makedirs(output_image_dir, exist_ok = True)
        # Duyệt qua các ảnh trong thư mục
        lr_image_files = os.listdir(lr_image_dir)
        hr_image_files = os.listdir(hr_image_dir)
        lr_image_files.sort()
        hr_image_files.sort()
    
        psnr_dict = {
            "mouse_bite" :[],
            "spur_":[], 
            "missing_hole":[],
            "short":[],
            "open_circuit":[],
            "spurious_copper":[]
    
        }
        ssim_dict = {
            "mouse_bite" :[],
            "spur_":[], 
            "missing_hole":[],
            "short":[],
            "open_circuit":[],
            "spurious_copper":[]
    
        }
        
        start = time.time()
        transform = transforms.ToTensor()
        with torch.no_grad():
            for lr_image_file, hr_image_file in tqdm(zip(lr_image_files, hr_image_files), unit = 'img'):
                # Đường dẫn đến ảnh
                
                lr_image_path = os.path.join(lr_image_dir, lr_image_file)
                hr_image_path = os.path.join(hr_image_dir, hr_image_file)
                output_image_path = os.path.join(output_image_dir, hr_image_file)
    
                # Tải và chuyển đổi ảnh
                lr_image = Image.open(lr_image_path).convert('RGB')
                hr_image = Image.open(hr_image_path).convert('RGB')
                hr_image = hr_image.resize((608, 608))
                lr_image = lr_image.resize((152, 152))
                if sub == 'IQE':
                    hr_image = hr_image.resize((152, 152))
                # lr_image = Image.open(lr_image_path).convert('RGB')
    
                lr_image = transform(lr_image).unsqueeze(0).to(device)  # Thêm batch dimension và chuyển sang GPU
                hr_image = transform(hr_image).unsqueeze(0).to(device)  # Thêm batch dimension và chuyển sang GPU
                # if model == bicubic:
                # Dự đoán
                if sub == 'ISR':
                    output = isr(lr_image)
                elif sub == 'IQE':
                    output = iqe(lr_image)
                elif sub == 'ISR + IQE':
                    output = iqe_sr(isr(lr_image))
                elif sub == 'IQE + ISR':
                    output = isr(iqe(lr_image))
                elif sub == 'IQE + ISR + IQE':
                    output = iqe_sr(isr_qe(iqe(lr_image)))
                elif sub == 'e2e':
                    output = iqe_e2e(isr(lr_image))
                elif sub == 'SRQE':
                    output = iqe_srqe(isr(lr_image))
                psnr,ssim = calculate_metrics(output, hr_image)
                for key in psnr_dict.keys():
                    if key in lr_image_path:
                        psnr_dict[key].append(psnr)
                        ssim_dict[key].append(ssim)
                        break

                # Chuyển đổi tensor đầu ra thành ảnh và lưu
                output_image = output.squeeze(0).to(device)  # Loại bỏ batch dimension và chuyển tensor sang CPU
                output_image = transforms.ToPILImage()(output_image)  # Chuyển tensor thành ảnh PIL
                output_image.save(output_image_path, )  # Lưu ảnh
        avg_psnr = [0, 0, 0, 0, 0, 0]
        avg_ssim = [0, 0, 0, 0, 0, 0]


        # Tính toán PSNR trung bình
        avg_psnr[0] = sum(psnr_dict['mouse_bite'])/len(psnr_dict['mouse_bite']) #mousebite_psnr
        avg_psnr[1] = sum(psnr_dict['spur_'])/len(psnr_dict['spur_']) #spur_psnr
        avg_psnr[2] = sum(psnr_dict['missing_hole'])/len(psnr_dict['missing_hole']) #missinghole_psnr 
        avg_psnr[3] = sum(psnr_dict['short'])/len(psnr_dict['short']) #short_psnr
        avg_psnr[4] = sum(psnr_dict['open_circuit'])/len(psnr_dict['open_circuit']) #opencircuit_psnr
        avg_psnr[5]= sum(psnr_dict['spurious_copper'])/len(psnr_dict['spurious_copper']) #spuriouscopper_psnr 
        average_psnr = sum(avg_psnr)/len(avg_psnr)

        avg_ssim[0] = sum(ssim_dict['mouse_bite'])/len(ssim_dict['mouse_bite']) #mousebite_ssim
        avg_ssim[1] = sum(ssim_dict['spur_'])/len(ssim_dict['spur_']) #spur_ssim
        avg_ssim[2] = sum(ssim_dict['missing_hole'])/len(ssim_dict['missing_hole']) #missinghole_ssim 
        avg_ssim[3] = sum(ssim_dict['short'])/len(ssim_dict['short']) #short_ssim
        avg_ssim[4] = sum(ssim_dict['open_circuit'])/len(ssim_dict['open_circuit']) #opencircuit_ssim
        avg_ssim[5]= sum(ssim_dict['spurious_copper'])/len(ssim_dict['spurious_copper']) #spuriouscopper_ssim  
        average_ssim = sum(avg_ssim)/len(avg_ssim)
        end = time.time()

        with open('runs/enhance_results.txt', 'a') as f:
            f.write(output_image_dir.split('/')[1] + '\n')
            f.write(f'missinghole_psnr: {avg_psnr[2]:.2f}' + '\n')
            f.write(f'mousebite_psnr: {avg_psnr[0]:.2f}' + '\n')
            f.write(f'opencircuit_psnr: {avg_psnr[4]:.2f}' + '\n')
            f.write(f'short_psnr: {avg_psnr[3]:.2f}' + '\n')
            f.write(f'spur_psnr: {avg_psnr[1]:.2f}' + '\n')
            f.write(f'spuriouscopper_psnr: {avg_psnr[5]:.2f}' + '\n')
            f.write(f'average_psnr: {average_psnr:.2f}' + '\n')
            f.write(f'time process: {end - start:.2f}' + '\n')

            f.write(f'missinghole_psnr: {avg_ssim[2]:.4f}' + '\n')
            f.write(f'mousebite_psnr: {avg_ssim[0]:.4f}' + '\n')
            f.write(f'opencircuit_psnr: {avg_ssim[4]:.4f}' + '\n')
            f.write(f'short_psnr: {avg_ssim[3]:.4f}' + '\n')
            f.write(f'spur_psnr: {avg_ssim[1]:.4f}' + '\n')
            f.write(f'spuriouscopper_psnr: {avg_ssim[5]:.4f}' + '\n')
            f.write(f'average_psnr: {average_ssim:.4f}' + '\n')
            f.write(f'time process: {end - start:.4f}' + '\n')
            f.write('\n')
            f.flush()
        source_dir = hr_label_path
    
        dest = f'output/enhance_{iqe_types}{imgsz}/test_{quality}_150_{sub}/labels'
    
        # Tạo thư mục đích nếu chưa tồn tại
        
        os.makedirs(dest, exist_ok=True)
    
        # Sao chép các file từ thư mục nguồn sang thư mục đích
        for filename in os.listdir(source_dir):
            source_file = os.path.join(source_dir, filename)
            file1 = os.path.join(dest, filename)
    
            shutil.copy(source_file, file1)
    
        # print("Đã sao chép các file thành công.")


Running e2e on quality 40...


0img [00:00, ?img/s]

0img [00:01, ?img/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 92.00 MiB. GPU 0 has a total capacity of 31.74 GiB of which 81.38 MiB is free. Process 30678 has 31.20 GiB memory in use. Process 85264 has 470.00 MiB memory in use. Of the allocated memory 81.17 MiB is allocated by PyTorch, and 20.83 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
import ultralytics
from ultralytics import YOLO
from PIL import ImageDraw, ImageFont
import yaml
import torch
import numpy as np
from tqdm import tqdm
from PIL import Image
import os
import matplotlib.pyplot as plt
import random 
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torch.utils.data import DataLoader, Dataset
from ultralytics.utils import ops
yolov8 = YOLO('exp/weights/bestyolov8.pt').model
yolov9 = YOLO()

yolov8_e2e = YOLO('exp/weights/bestyolov8.pt').model
yolov8_e2e.load_state_dict(torch.load('exp/Enhancer_Small_e2e_64/best_detection_weight.pth')['model_state_dict'])
yolov8_srqe = YOLO('exp/weights/bestyolov8.pt').model
yolov8_srqe.load_state_dict(torch.load('exp/YOLOv8_SRQE_fintune/best_detection_weight.pth')['model_state_dict'])



/home/u9564043/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>

In [8]:
from PIL import ImageDraw, ImageFont
import yaml
import torch
import numpy as np
from tqdm import tqdm
from PIL import Image
import os
import matplotlib.pyplot as plt
import random 
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torch.utils.data import DataLoader, Dataset
from ultralytics.utils import ops



class_names = {
    0: 'mouse_bite',
    1: 'spur',
    2: 'missing_hole',
    3: 'short',
    4: 'open_circuit',
    5: 'spurious_copper'
}
label_colors = {
    0: (255, 0, 0),  # Red
    1: (0, 255, 0),  # Green
    2: (0, 0, 255),  # Blue
    3: (255, 255, 0),  # Yellow
    4: (255, 0, 255),  # Magenta
    5: (0, 255, 255)   # Cyan
}
def draw_and_save_predictions(image, boxes, labels, scores, class_names, save_path=None, font_path=None):
    """
    Vẽ bounding box, nhãn và độ tự tin lên ảnh, sau đó lưu ảnh nếu cần.
    
    Args:
        image (PIL.Image.Image): Ảnh đầu vào.
        boxes (torch.Tensor): Tensor chứa các bounding box, định dạng [x1, y1, x2, y2].
        labels (torch.Tensor): Tensor chứa nhãn các bounding box.
        scores (torch.Tensor): Tensor chứa điểm tự tin của các bounding box.
        class_names (dict): Mapping từ ID nhãn sang tên lớp.
        save_path (str): Đường dẫn để lưu ảnh (nếu không truyền, ảnh sẽ không được lưu).
        font_path (str): Đường dẫn tới file font TrueType (nếu không truyền sẽ dùng font mặc định).
    
    Returns:
        PIL.Image.Image: Ảnh đã được vẽ bounding box.
    """
    # Tạo bản sao ảnh để vẽ
    draw_image = image.copy()
    draw = ImageDraw.Draw(draw_image)
    
    # Tải font (nếu có)
    if font_path:
        try:
            font = ImageFont.truetype(font_path, size=20)
        except Exception as e:
            print(f"Không thể tải font từ {font_path}. Sử dụng font mặc định.")
            font = ImageFont.load_default()
    else:
        font = ImageFont.load_default()
    
    # Vẽ từng bounding box
    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box
        label_text = f"{class_names.get(label.item(), 'Unknown')} {score:.2f}"
        color = label_colors.get(label.item())
        # Vẽ hình chữ nhật
        draw.rectangle([x1, y1, x2, y2], outline=color, width=2)
        
        # Vẽ nhãn với nền
        if hasattr(draw, "textbbox"):
            text_bbox = draw.textbbox((x1, y1), label_text, font=font)
            text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
        # else:
            # text_width, text_height = draw.textsize(label_text, font=font)
        draw.rectangle(
            [x1, y1 - text_height, x1 + text_width, y1],
            fill=color
        )
        draw.text((x1, y1 - text_height), label_text, fill="white", font=font)
    
    # Lưu ảnh nếu `save_path` được cung cấp
    if save_path:
        draw_image.save(save_path)
        # plt.imshow(draw_image)
        # print(f"Ảnh đã được lưu tại: {save_path}")



def yolo_to_xyxy(bboxes, img_w, img_h):
    """
    Convert YOLO-format [cx, cy, w, h] normalized -> [x1, y1, x2, y2] absolute
    """
    cx, cy, w, h = bboxes[:, 0], bboxes[:, 1], bboxes[:, 2], bboxes[:, 3]
    x1 = (cx - w/2) * img_w
    y1 = (cy - h/2) * img_h
    x2 = (cx + w/2) * img_w
    y2 = (cy + h/2) * img_h
    return torch.stack([x1, y1, x2, y2], dim=1)

def run_inference(image, model):
    results = model.predict(image, verbose=False)
    predictions = results[0].boxes
    # Convert to numpy for WBF compatibility
    boxes = predictions.xyxy.cpu().numpy()
    scores = predictions.conf.cpu().numpy()
    labels = predictions.cls.cpu().numpy()
    return boxes, scores, labels

def normalize_boxes(boxes, image_size):
    """Normalize box coordinates to [0, 1] range"""
    width, height = image_size
    normalized_boxes = boxes.copy()
    normalized_boxes[:, [0, 2]] /= width
    normalized_boxes[:, [1, 3]] /= height
    return normalized_boxes

def denormalize_boxes(boxes, image_size):
    """Convert normalized boxes back to pixel coordinates"""
    width, height = image_size
    denormalized_boxes = boxes.copy()
    denormalized_boxes[:, [0, 2]] *= width
    denormalized_boxes[:, [1, 3]] *= height
    return denormalized_boxes

def ensemble_inference(image, method):
    # Get predictions from both models
    boxes_yolov8, scores_yolov8, labels_yolov8 = run_inference(image, yolov8)
    boxes_yolov9, scores_yolov9, labels_yolov9 = run_inference(image, yolov9)
    # print(boxes_yolov8)
    # Get image size
    if isinstance(image, torch.Tensor):
        height, width = image.shape[-2:]
    else:  # PIL Image
        width, height = image.size
    image_size = (width, height)
    if method == 'yolov8':
        boxes, scores, labels = boxes_yolov8, scores_yolov8, labels_yolov8
    else:
        boxes, scores, labels = boxes_yolov9, scores_yolov9, labels_yolov9
   
    # Convert back to torch tensors
    boxes = torch.from_numpy(boxes).float()
    scores = torch.from_numpy(scores).float()
    labels = torch.from_numpy(labels).long()
    return boxes, scores, labels

def read_label_file(label_path, img_width, img_height):
    boxes = []
    labels = []
    with open(label_path, "r") as f:
        for line in f:
            box = list(map(float, line.strip().split()))
            labels.append(int(box[0]))
            boxes.append(yolo_to_xyxy(box, img_width, img_height))
    return boxes, labels

log_fp = open('runs/detect_results.txt', 'a')
# iqe_types = 'small'
# imgsz = 64
# detect_subs = ['_IQE', '_ISR + IQE', '_IQE + ISR', '_IQE + ISR + IQE', '_e2e']
for quality in test_quality:

    # for sub in ['','_ISR','_IQE', '_ISR + IQE', '_IQE + ISR', '_IQE + ISR + IQE', 'full']:
    # for sub in ['','_ISR', '_IQE', '_ISR + IQE', '_IQE + ISR', 'full']:
    # for sub in ['_IQE + ISR + IQE' ]:
    for sub in ['_e2e', '_SRQE' ]:
    # for sub in detect_subs:
        # Tạo thư mục gốc cho inference
        base_output_dir = "runs"
        os.makedirs(base_output_dir, exist_ok=True)
        path = f'output/enhance_{iqe_types}{imgsz}/test_{quality}_150{sub}'
        # path = '/kaggle/input/test600/test1_600x600'

        if sub == 'full':
            path = f'output/test_{quality}_600' 
        data = {
            'train': f'/kaggle/input/tesstt/images',
            'val': f'{path}/images',
            'nc': 6,
            'names': {
                0: 'mouse_bite',
                1: 'spur',
                2: 'missing_hole',
                3: 'short',
                4: 'open_circuit',
                5: 'spurious_copper'
            }
        }
        
        with open('output/data.yaml', 'w') as file:
            yaml.dump(data, file, default_flow_style=False)
    
        print(f"Evaluating {path} dataset at QF {quality}...")
        log_fp.write(f"Evaluating {path} dataset at QF {quality}..\n")
        log_fp.flush()

        # # Run individual model validation
        # results1 = yolov8.val(data='output/data.yaml', batch=1)
        # results2 = yolov9.val(data='output/data.yaml', batch=1)
    
        # path = f'output/test_{quality}_150{sub}'
        # path = '/kaggle/input/test600/test1_600x600'
        images_dir = f"{path}/images"
        images_files = sorted(os.listdir(images_dir))
        labels_dir = f"{path}/labels"
        labels_files = sorted(os.listdir(labels_dir))

        valid_dataset = YOLOTestDataset(images_dir, labels_dir)
        valid_loader = DataLoader(valid_dataset, collate_fn=yolo_collate_fn)
        # Initialize Mean Average Precision metric
        map_metric = MeanAveragePrecision(iou_thresholds=[0.5], iou_type="bbox", class_metrics=True)
        
        # for method in ['yolov8', 'yolov9',  'nms', 'soft_nms','nmw','wbf']:
        if sub == '_e2e':
            detection = yolov8_e2e
        elif sub == '_SRQE':
            detection = yolov8_srqe
        else:
            detection = yolov8
        detection.to(device)
        detection.eval()
        map_metric.reset()
        print(f"Running inference with on {sub}...")
        # Tạo thư mục riêng cho từng phương pháp
        method_output_dir = os.path.join(base_output_dir, sub)
        os.makedirs(method_output_dir, exist_ok=True)

        pbar = tqdm(valid_loader, desc=f'Val on {sub}', unit='batch', leave=False)
        for i, (hr_images, labels) in enumerate(pbar):
            hr_images = hr_images.to(device)  # (B T C H W)s

            pred = detection(hr_images)    

            preds_for_metric = ops.non_max_suppression(pred,
                                                       conf_thres=0.25, # low conf for mAP
                                                       iou_thres=0.5,
                                                       agnostic=False,
                                                       max_det=300,
                                                       nc=data['nc'])
            predictions, targets = post_process(preds_for_metric, labels, 608, 608)             
            # print(f'pred: {predictions}\nlabels: {targets}')

            map_metric.update(predictions, targets)    
            # Vẽ và lưu ảnh
            # flag = bool(random.randint(0, 1))
            # if flag:
            save_path = os.path.join(method_output_dir, images_files[i])  # Lưu với cùng tên ảnh
            draw_and_save_predictions(
                image=transforms.ToPILImage()(hr_images.squeeze(0).cpu()).copy(),
                boxes=predictions[0]['boxes'].numpy(),
                labels=predictions[0]['labels'].numpy(),
                scores=predictions[0]['scores'].numpy(),
                class_names=data['names'],
                save_path=save_path
            )

        # Tính toán kết quả sau khi xử lý tất cả ảnh
        results = map_metric.compute()
        print(f"\nResults for on {sub}:")
        log_fp.write(f"\nResults for on {sub}:\n")
        map50_per_class = results['map_per_class']
        for class_idx, map_value in enumerate(map50_per_class):
            print(f"Class {class_idx}: mAP@50 = {map_value.item():.3f}")
            log_fp.write(f"Class {class_idx}: mAP@50 = {map_value.item():.3f}\n")
            log_fp.flush()
        print(results['map'])
        log_fp.write(f"Average: {results['map']:.3f}\n")
        log_fp.flush()
        print("-" * 50)


Evaluating output/enhance_small64/test_40_150_e2e dataset at QF 40...
Running inference with on _e2e...


Val on _e2e:   0%|          | 0/1068 [00:00<?, ?batch/s]


Results for on _e2e:
Class 0: mAP@50 = 0.727
Class 1: mAP@50 = 0.683
Class 2: mAP@50 = 0.975
Class 3: mAP@50 = 0.757
Class 4: mAP@50 = 0.833
Class 5: mAP@50 = 0.728
tensor(0.7838)
--------------------------------------------------
Evaluating output/enhance_small64/test_40_150_SRQE dataset at QF 40...
Running inference with on _SRQE...



Results for on _SRQE:
Class 0: mAP@50 = 0.768
Class 1: mAP@50 = 0.643
Class 2: mAP@50 = 0.934
Class 3: mAP@50 = 0.694
Class 4: mAP@50 = 0.822
Class 5: mAP@50 = 0.664
tensor(0.7542)
--------------------------------------------------


In [ ]:
# from PIL import ImageDraw, ImageFont
# import yaml
# import torch
# import numpy as np
# from ensemble_boxes import *
# from tqdm import tqdm
# from PIL import Image
# import os
# import matplotlib.pyplot as plt
# import random 
# from torchmetrics.detection.mean_ap import MeanAveragePrecision


# class_names = {
#     0: 'mouse_bite',
#     1: 'spur',
#     2: 'missing_hole',
#     3: 'short',
#     4: 'open_circuit',
#     5: 'spurious_copper'
# }
# label_colors = {
#     0: (255, 0, 0),  # Red
#     1: (0, 255, 0),  # Green
#     2: (0, 0, 255),  # Blue
#     3: (255, 255, 0),  # Yellow
#     4: (255, 0, 255),  # Magenta
#     5: (0, 255, 255)   # Cyan
# }
# def draw_and_save_predictions(image, boxes, labels, scores, class_names, save_path=None, font_path=None):
#     """
#     Vẽ bounding box, nhãn và độ tự tin lên ảnh, sau đó lưu ảnh nếu cần.
    
#     Args:
#         image (PIL.Image.Image): Ảnh đầu vào.
#         boxes (torch.Tensor): Tensor chứa các bounding box, định dạng [x1, y1, x2, y2].
#         labels (torch.Tensor): Tensor chứa nhãn các bounding box.
#         scores (torch.Tensor): Tensor chứa điểm tự tin của các bounding box.
#         class_names (dict): Mapping từ ID nhãn sang tên lớp.
#         save_path (str): Đường dẫn để lưu ảnh (nếu không truyền, ảnh sẽ không được lưu).
#         font_path (str): Đường dẫn tới file font TrueType (nếu không truyền sẽ dùng font mặc định).
    
#     Returns:
#         PIL.Image.Image: Ảnh đã được vẽ bounding box.
#     """
#     # Tạo bản sao ảnh để vẽ
#     draw_image = image.copy()
#     draw = ImageDraw.Draw(draw_image)
    
#     # Tải font (nếu có)
#     if font_path:
#         try:
#             font = ImageFont.truetype(font_path, size=20)
#         except Exception as e:
#             print(f"Không thể tải font từ {font_path}. Sử dụng font mặc định.")
#             font = ImageFont.load_default()
#     else:
#         font = ImageFont.load_default()
    
#     # Vẽ từng bounding box
#     for box, label, score in zip(boxes, labels, scores):
#         x1, y1, x2, y2 = box
#         label_text = f"{class_names.get(label.item(), 'Unknown')} {score:.2f}"
#         color = label_colors.get(label.item())
#         # Vẽ hình chữ nhật
#         draw.rectangle([x1, y1, x2, y2], outline=color, width=2)
        
#         # Vẽ nhãn với nền
#         if hasattr(draw, "textbbox"):
#             text_bbox = draw.textbbox((x1, y1), label_text, font=font)
#             text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
#         # else:
#             # text_width, text_height = draw.textsize(label_text, font=font)
#         draw.rectangle(
#             [x1, y1 - text_height, x1 + text_width, y1],
#             fill=color
#         )
#         draw.text((x1, y1 - text_height), label_text, fill="white", font=font)
    
#     # Lưu ảnh nếu `save_path` được cung cấp
#     if save_path:
#         draw_image.save(save_path)
#         # plt.imshow(draw_image)
#         # print(f"Ảnh đã được lưu tại: {save_path}")



# def yolo_to_xyxy(box, img_width, img_height):
#     cx, cy, w, h = box[1:]
#     x1 = (cx - w / 2) * img_width
#     y1 = (cy - h / 2) * img_height
#     x2 = (cx + w / 2) * img_width
#     y2 = (cy + h / 2) * img_height
#     return [x1, y1, x2, y2]

# def run_inference(image, model):
#     results = model.predict(image, verbose=False)
#     predictions = results[0].boxes
#     # Convert to numpy for WBF compatibility
#     boxes = predictions.xyxy.cpu().numpy()
#     scores = predictions.conf.cpu().numpy()
#     labels = predictions.cls.cpu().numpy()
#     return boxes, scores, labels

# def normalize_boxes(boxes, image_size):
#     """Normalize box coordinates to [0, 1] range"""
#     width, height = image_size
#     normalized_boxes = boxes.copy()
#     normalized_boxes[:, [0, 2]] /= width
#     normalized_boxes[:, [1, 3]] /= height
#     return normalized_boxes

# def denormalize_boxes(boxes, image_size):
#     """Convert normalized boxes back to pixel coordinates"""
#     width, height = image_size
#     denormalized_boxes = boxes.copy()
#     denormalized_boxes[:, [0, 2]] *= width
#     denormalized_boxes[:, [1, 3]] *= height
#     return denormalized_boxes

# def ensemble_inference(image, method):
#     # Get predictions from both models
#     boxes_yolov8, scores_yolov8, labels_yolov8 = run_inference(image, yolov8)
#     boxes_yolov9, scores_yolov9, labels_yolov9 = run_inference(image, yolov9)
#     # print(boxes_yolov8)
#     # Get image size
#     if isinstance(image, torch.Tensor):
#         height, width = image.shape[-2:]
#     else:  # PIL Image
#         width, height = image.size
#     image_size = (width, height)
#     if method in ['yolov8', 'yolov9']:
#         if method == 'yolov8':
#             boxes, scores, labels = boxes_yolov8, scores_yolov8, labels_yolov8
#         else:
#             boxes, scores, labels = boxes_yolov9, scores_yolov9, labels_yolov9
#     else:
#         boxes_yolov8_norm = normalize_boxes(boxes_yolov8, image_size)
#         boxes_yolov9_norm = normalize_boxes(boxes_yolov9, image_size)
#         boxes_list = [boxes_yolov8_norm, boxes_yolov9_norm]
#         scores_list = [scores_yolov8, scores_yolov9]
#         labels_list = [labels_yolov8, labels_yolov9]
#         if len(boxes_list) == 0 or all(len(b) == 0 for b in boxes_list):
#             # print("Warning: No valid boxes detected. Returning empty results.")
#             boxes = np.empty((0, 4), dtype=np.float32)  # Dạng bounding box
#             scores = np.empty((0,), dtype=np.float32)  # Dạng scores
#             labels = np.empty((0,), dtype=np.int32)
#         else:
#             # print(boxes_list, scores_list)
#             if method == 'wbf':
#                 # Apply Weighted Box Fusion
#                 boxes, scores, labels = weighted_boxes_fusion(
#                     boxes_list,
#                     scores_list,
#                     labels_list,
#                     weights=[1, 2],  # Equal weights for both models
#                     iou_thr=0.5,
#                     skip_box_thr=0.0001,
#                 )
#             elif method == 'nms':
#                 boxes, scores, labels = nms(
#                     boxes_list,
#                     scores_list,
#                     labels_list,
#                     weights=[1, 2],  # Equal weights for both models
#                     iou_thr=0.5
#                 )
#                 # print(type(boxes), '\n', type(scores), '\n',type(labels))
#             elif method == 'soft_nms':
#                 boxes, scores, labels = soft_nms(
#                     boxes_list,
#                     scores_list,
#                     labels_list,
#                     weights=[1, 2],  # Equal weights for both models
#                     iou_thr=0.5,
#                     sigma = 0.1,
#                     thresh = 0.0001
#                 )
#             else:
#                 boxes, scores, labels = non_maximum_weighted(
#                     boxes_list,
#                     scores_list,
#                     labels_list,
#                     weights=[1, 2],  # Equal weights for both models
#                     iou_thr=0.5,
#                     skip_box_thr=0.0001,
#                 )
#             boxes = denormalize_boxes(boxes, image_size)
#     # Convert back to torch tensors
#     boxes = torch.from_numpy(boxes).float()
#     scores = torch.from_numpy(scores).float()
#     labels = torch.from_numpy(labels).long()
#     return boxes, scores, labels

# def read_label_file(label_path, img_width, img_height):
#     boxes = []
#     labels = []
#     with open(label_path, "r") as f:
#         for line in f:
#             box = list(map(float, line.strip().split()))
#             labels.append(int(box[0]))
#             boxes.append(yolo_to_xyxy(box, img_width, img_height))
#     return boxes, labels

# # Main validation loop
# # for sub in ['LR', 'Bicubic', 'SRCNN', 'VDSR', 'SRResNet', 'EDSR', 'HQSR_S', 'HQSR_C', 'HR']:
# # for sub in ['Bicubic']:
# # for sub in ['SRCNN', 'VDSR', 'SRResNet', 'EDSR', 'HQSR_S', 'HQSR_C', 'HR']:
# # for sub in ['test_150', 'test_150_ESR_Canny','test_150_ESR_Sobel','test_600']:
# # for quality in [10, 20, 40, 80]:
# # for quality in [10, 20, 30, 40]:
# # for quality in [20, 40, 80]:

# for quality in test_quality:

#     # for sub in ['','_ISR','_IQE', '_ISR + IQE', '_IQE + ISR', '_IQE + ISR + IQE', 'full']:
#     # for sub in ['','_ISR', '_IQE', '_ISR + IQE', '_IQE + ISR', 'full']:
#     # for sub in ['_IQE + ISR + IQE' ]:
#     # for sub in ['','_ISR' ]:
#     for sub in detect_subs:
#         # Tạo thư mục gốc cho inference
#         base_output_dir = "runs"
#         os.makedirs(base_output_dir, exist_ok=True)
#         path = f'output/test_{quality}_150{sub}'
#         # path = 'output/test_600'

#         if sub == 'full':
#             path = f'output/test_{quality}_600' 
#         data = {
#             'train': f'/kaggle/input/tesstt/images',
#             'val': f'{path}/images',
#             'nc': 6,
#             'names': {
#                 0: 'mouse_bite',
#                 1: 'spur',
#                 2: 'missing_hole',
#                 3: 'short',
#                 4: 'open_circuit',
#                 5: 'spurious_copper'
#             }
#         }
        
#         with open('output/data.yaml', 'w') as file:
#             yaml.dump(data, file, default_flow_style=False)
    
#         print(f"Evaluating {sub} dataset at QF {quality}...")
#         # # Run individual model validation
#         # results1 = yolov8.val(data='output/data.yaml', batch=1)
#         # results2 = yolov9.val(data='output/data.yaml', batch=1)
    
#         # path = f'output/test_{quality}_150{sub}'
#         # path = '/kaggle/input/test600/test1_600x600'
#         images_dir = f"{path}/images"
#         labels_dir = f"{path}/labels"
#         # Initialize Mean Average Precision metric
#         map_metric = MeanAveragePrecision(iou_thresholds=[0.5], iou_type="bbox", class_metrics=True)
        
#         # for method in ['yolov8', 'yolov9',  'nms', 'soft_nms','nmw','wbf']:
#         for method in ['yolov8']:
#             map_metric.reset()
#             print(f"Running {method} inference with on {sub}...")
#             # Tạo thư mục riêng cho từng phương pháp
#             method_output_dir = os.path.join(base_output_dir, method, sub)
#             os.makedirs(method_output_dir, exist_ok=True)
    
#             for image_file in tqdm(os.listdir(images_dir), unit='img'):
#                 img_path = os.path.join(images_dir, image_file)
#                 image = Image.open(img_path).convert('RGB')
#                 # Get true labels
#                 label_path = os.path.join(labels_dir, image_file[:-4] + ".txt")
#                 true_boxes, true_labels = read_label_file(label_path, image.size[0], image.size[1])
#                 true_boxes = torch.tensor(true_boxes, dtype=torch.float32)
#                 true_labels = torch.tensor(true_labels, dtype=torch.int64)
    
#                 # Chạy inference theo phương pháp
#                 pred_boxes, pred_scores, pred_labels = ensemble_inference(image, method)
    
#                 # Format predictions and targets for metric update
#                 predictions = [{
#                     'boxes': pred_boxes,
#                     'scores': pred_scores,
#                     'labels': pred_labels
#                 }]
                
#                 targets = [{
#                     'boxes': true_boxes,
#                     'labels': true_labels
#                 }]
#                 # print(predictions,'\n', targets)
#                 map_metric.update(predictions, targets)
    
#                 # Vẽ và lưu ảnh
#                 # flag = bool(random.randint(0, 1))
#                 # if flag:
#                 save_path = os.path.join(method_output_dir, image_file)  # Lưu với cùng tên ảnh
#                 draw_and_save_predictions(
#                     image=image.copy(),
#                     boxes=pred_boxes.numpy(),
#                     labels=pred_labels.numpy(),
#                     scores=pred_scores.numpy(),
#                     class_names=data['names'],
#                     save_path=save_path
#                 )
    
#             # Tính toán kết quả sau khi xử lý tất cả ảnh
#             results = map_metric.compute()
#             print(f"\nResults for {method} on {sub}:")
#             map50_per_class = results['map_per_class']
#             for class_idx, map_value in enumerate(map50_per_class):
#                 print(f"Class {class_idx}: mAP@50 = {map_value.item():.3f}")
#             print(results['map'])
#             print("-" * 50)


Evaluating _IQE dataset at QF 40...
Running yolov8 inference with on _IQE...


  1%|          | 10/1068 [00:00<00:21, 49.25img/s]

100%|██████████| 1068/1068 [00:19<00:00, 53.84img/s]



Results for yolov8 on _IQE:
Class 0: mAP@50 = 0.125
Class 1: mAP@50 = 0.038
Class 2: mAP@50 = 0.265
Class 3: mAP@50 = 0.226
Class 4: mAP@50 = 0.270
Class 5: mAP@50 = 0.108
tensor(0.1721)
--------------------------------------------------
Evaluating _ISR + IQE dataset at QF 40...
Running yolov8 inference with on _ISR + IQE...


100%|██████████| 1068/1068 [00:28<00:00, 37.08img/s]


...


Results for yolov8 on _IQE + ISR:
Class 0: mAP@50 = 0.231
Class 1: mAP@50 = 0.085
Class 2: mAP@50 = 0.762
Class 3: mAP@50 = 0.226
Class 4: mAP@50 = 0.387
Class 5: mAP@50 = 0.388
tensor(0.3466)
--------------------------------------------------
Evaluating _IQE + ISR + IQE dataset at QF 40...
Running yolov8 inference with on _IQE + ISR + IQE...


100%|██████████| 1068/1068 [00:28<00:00, 37.96img/s]



Results for yolov8 on _IQE + ISR + IQE:
Class 0: mAP@50 = 0.226
Class 1: mAP@50 = 0.110
Class 2: mAP@50 = 0.752
Class 3: mAP@50 = 0.223
Class 4: mAP@50 = 0.365
Class 5: mAP@50 = 0.381
tensor(0.3427)
--------------------------------------------------
Evaluating _e2e dataset at QF 40...
Running yolov8 inference with on _e2e...


100%|██████████| 1068/1068 [00:30<00:00, 35.45img/s]



Results for yolov8 on _e2e:
Class 0: mAP@50 = 0.072
Class 1: mAP@50 = 0.039
Class 2: mAP@50 = 0.036
Class 3: mAP@50 = 0.020
Class 4: mAP@50 = 0.058
Class 5: mAP@50 = 0.016
tensor(0.0400)
--------------------------------------------------
